# comercio_ext_auxiliares.tb_referencia_uf_mun
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/UF_MUN_delta`
## 📌 Descrição do arquivo
Referência de **Municípios** com código geográfico e UF.
|Coluna|Descrição|
|---|---|
|`CO_MUN_GEO`|Código geográfico do município|
|`NO_MUN`|Nome do município|
|`NO_MUN_MIN`|Nome (minúsculo/padronizado)|
|`SG_UF`|Sigla da UF|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:
val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/UF_MUN_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_auxiliares/tb_referencia_uf_mun/"   
val silverTable =    "tb_referencia_uf_mun"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_MUN_GEO" , StringType ,  nullable = false ) ,
      StructField ( "NO_MUN" , StringType ,  nullable = false ) ,
      StructField ( "NO_MUN_MIN" , StringType ,  nullable = false ) ,
      StructField ( "SG_UF" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_MUN_GEO", trim(upper(col("CO_MUN_GEO").cast(StringType))))
  .withColumn("NO_MUN", col("NO_MUN").cast(StringType))
  .withColumn("NO_MUN_MIN", col("NO_MUN_MIN").cast(StringType))
  .withColumn("SG_UF", trim(upper(col("SG_UF").cast(StringType))))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/UF_MUN_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_MUN_GEO").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_MUN_GEO"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_MUN_GEO"), col("NO_MUN"), col("NO_MUN_MIN"), col("SG_UF"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}